In [ ]:
import pandas as pd
import datetime

In [ ]:
caminho_do_arquivo = "https://raw.githubusercontent.com/Russel-Servicos/automacao-mkt-chat/main/planilha_chat_estagio_1de2_exemplo.csv"

df = pd.read_csv(caminho_do_arquivo)

data = "11/09/2024"

In [ ]:
# deixar unico os canais, origens, meios e campanhas, fazer o calculo com todos
# usar comprehensive list para preencher o resto das linhas que falta para adicionar na coluna

In [ ]:
## Conta quantidade de qualificado, não qualificado e emprego
qty_ids = len(df.index)

qty_qualified = df.query("Qualificação == 'Qualificado'")
qty_qualified = len(qty_qualified.index)
percentage_qualified = qty_qualified / qty_ids
percentage_qualified = "{:.2f}".format(percentage_qualified)
percentage_qualified = float(percentage_qualified)

qty_not_qualified = df.query("Qualificação == 'Não Qualificado'")
qty_not_qualified = len(qty_not_qualified.index)
percentage_not_qualified = qty_not_qualified / qty_ids
percentage_not_qualified = "{:.2f}".format(percentage_not_qualified)
percentage_not_qualified = float(percentage_not_qualified)

qty_emprego = df.query("Qualificação == 'Emprego'")
qty_emprego = len(qty_emprego.index)
percentage_emprego = qty_emprego / qty_ids
percentage_emprego = "{:.2f}".format(percentage_emprego)
percentage_emprego = float(percentage_emprego)

In [ ]:
#
unique = pd.unique(df["Campanha"])
unique
# remover nan da list

# pega total de linhas, cria lista de contagem e taxa de conversao com valores e nan, joga na coluna
df.shape

(89, 13)

In [ ]:
## Contagem e Taxa de Conversão das colunas: Canal, Origem, Meio e Campanha
def generate_data_column_contagem(values, column):
  result_list = []

  for value in values:
    df_filtered = df.query(f"{column} == '{value}'")
    total_rows = len(df_filtered.index)
    df_qualified = df_filtered.query("Qualificação == 'Qualificado'")
    qty_qualified = len(df_qualified.index)
    calculation = f"{value}: {total_rows} - {qty_qualified}"

    if total_rows == 0:
      percentage = 0.0
    else:
      percentage = qty_qualified / total_rows

    if percentage == 0.0:
      percentage = 0
    else:
      percentage = "{:.0%}".format(percentage)

    result_list.append({
        "TaxaConversão": percentage,
        "Calculo": calculation
    })
  return result_list

canais = ["Adwords", "Direct", "Search Engine", "Referrals", "WhatsApp"]
origens = ["google"]
meios = ["display", "pesquisa", "youtube"]
campanhas = ['aeroporto', 'barerestaurante', 'condominio', 'construcao', 'construcaorj', 'estacionamento', 'geral', 'logistica', 'lp-construcao', 'opmaquinas', 'pacotelimpeza', 'pacotelimpezarj', 'saude', 'terceirizacao', 'varejo', 'varejorj', 'ti', 'ygeral', 'yremarketing']

result_canais = generate_data_column_contagem(values=canais, column="Canal")
result_origens = generate_data_column_contagem(values=origens, column="Origem")
result_meios = generate_data_column_contagem(values=meios, column="Meio")
result_campanhas = generate_data_column_contagem(values=campanhas, column="Campanha")

result_campanhas

[{'TaxaConversão': 0, 'Calculo': 'aeroporto: 0 - 0'},
 {'TaxaConversão': '33%', 'Calculo': 'barerestaurante: 3 - 1'},
 {'TaxaConversão': 0, 'Calculo': 'condominio: 0 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'construcao: 5 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'construcaorj: 0 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'estacionamento: 2 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'geral: 0 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'logistica: 2 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'lp-construcao: 0 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'opmaquinas: 3 - 0'},
 {'TaxaConversão': '38%', 'Calculo': 'pacotelimpeza: 8 - 3'},
 {'TaxaConversão': 0, 'Calculo': 'pacotelimpezarj: 1 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'saude: 0 - 0'},
 {'TaxaConversão': '50%', 'Calculo': 'terceirizacao: 2 - 1'},
 {'TaxaConversão': 0, 'Calculo': 'varejo: 3 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'varejorj: 1 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'ti: 2 - 0'},
 {'TaxaConversão': 0, 'Calculo': 'ygeral: 3 - 0'},
 {'

In [ ]:
## Insere valores na coluna Contagem e Taxa de Conversão
all_result_columns = result_canais + [ {} ] + result_origens + [ {} ] + result_meios + [ {} ] + result_campanhas

list_taxa_conversao = []
list_contagem = []

for value in all_result_columns:
  dict_is_empty = len(value.keys()) == 0

  if dict_is_empty:
    list_taxa_conversao.append("")
    list_contagem.append("")
  else:
    list_taxa_conversao.append(value["TaxaConversão"])
    list_contagem.append(value["Calculo"])


# df.iloc[:, "Taxa de Conversão"] = list_taxa_conversao
df
# df.iloc[:, "Contagem"] = list_contagem

df2 = pd.DataFrame([[1, 2], [4, 5], [7, 8], [9, 10], [11, 23]])
df2.iloc[1:5,]
# df2

,0,1
1,4,5
2,7,8
3,9,10
4,11,23


In [ ]:
## Adiciona o valor Chat SalesIQ na coluna Canal
df_salesiq = df.query("Canal in ['Adwords', 'Direct', 'Referrals', 'Search Engine']")
df_salesiq.loc[:, "Canal"] = "Chat SalesIQ"

df_not_salesiq = df.query("Canal not in ['Adwords', 'Direct', 'Referrals', 'Search Engine']")

df_final = pd.concat([df_salesiq, df_not_salesiq])

df_final

<class 'pandas.core.frame.DataFrame'>
Index: 89 entries, 1 to 79
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Data                    89 non-null     object 
 1   ID do Visitante         88 non-null     object 
 2   Canal                   88 non-null     object 
 3   Página de Aterrissagem  76 non-null     object 
 4   Question                88 non-null     object 
 5   Origem                  39 non-null     object 
 6   Meio                    38 non-null     object 
 7   Campanha                38 non-null     object 
 8   Estado                  76 non-null     object 
 9   Keyword                 34 non-null     object 
 10  Qualificação            88 non-null     object 
 11  Contagem                0 non-null      float64
 12  Taxa de Conversão       0 non-null      float64
dtypes: float64(2), object(11)
memory usage: 9.7+ KB


In [ ]:
## Adiciona valores totais na primeira linha, embaixo da linha das colunas
df_final.loc[-1] = [data, qty_ids, qty_qualified, percentage_qualified, "", "",
              "", "", "", "", "", "", ""]
df_final.index = df_final.index + 1
df_final = df_final.sort_index()

In [ ]:
# Exportar como CSV
now = datetime.datetime.now()
now = now.strftime("%Y_%m_%d_%H_%M")
df_final.to_csv(f'./planilha_chat_estagio_2de2_{now}.csv', index=False)